# In-Core Computation of Geometric Centralities with HyperBall: A Hundred Billion Nodes and Beyond



In [55]:
import numpy as np

In [138]:
# class Node:
#     def __init__(self,val):
#         self.value = val
#     def __hash__(self):
#         return hash(self.value)
        
# class Edge:
#     def __init__(self, Node_A, Node_B): #A->B
#         self.origin = Node_A
#         self.destination = Node_B
#     def __hash__(self):
#         return hash(tuple((self.origin,self.destination)))

class Graph:
    def __init__(self):
        self.NodesSet=set({})
        self.EdgesDict={}
        
    def add_Edge(self, origin : int, destination : int):
        self.EdgesDict[origin].add(destination)
        
    def add_Node(self, node : int):
        self.NodesSet.add(node)
        self.EdgesDict[node]=set({})
    
    def size(self):
        '''return (number of nodes, number of edges)'''
        return (len(self.NodesSet),np.sum([len(a) for a in self.EdgesDict.values()]))

[10,
 0,
 0,
 10,
 10,
 10,
 0,
 9,
 10,
 10,
 0,
 10,
 0,
 0,
 10,
 0,
 1,
 10,
 0,
 0,
 0,
 9,
 0,
 0,
 0,
 0,
 9,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 9,
 10,
 8,
 10,
 10,
 9,
 0,
 9,
 9,
 9,
 9,
 0,
 10,
 0,
 0,
 0,
 10,
 10,
 10,
 9,
 10,
 9,
 0,
 0,
 8,
 10,
 9,
 0,
 10,
 10,
 10,
 9,
 10,
 0,
 1,
 1,
 10,
 0,
 9,
 0,
 0,
 10,
 10,
 0,
 0,
 10,
 10,
 10,
 8,
 10,
 8,
 0,
 0,
 10,
 4,
 0,
 0,
 9,
 0,
 0,
 0,
 10,
 9,
 10,
 0,
 0,
 10,
 2,
 0,
 1,
 0,
 0,
 10,
 0,
 0,
 10,
 0,
 10,
 0,
 0,
 1,
 0,
 10,
 3,
 3,
 10,
 10,
 9,
 10,
 0,
 0,
 0,
 0,
 9,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 10,
 2,
 10,
 10,
 10,
 10,
 0,
 0,
 0,
 10,
 10,
 0,
 0,
 0,
 0,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 10,
 0,
 0,
 10,
 0,
 9,
 10,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 10,
 6,
 0,
 10,
 0,
 0,
 10,
 0,
 0,
 10,
 0,
 9,
 0,
 9,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 9,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 10,
 0,
 0,
 0,
 0,
 9,
 10,
 9,
 9,
 10,
 10,
 0,
 0,
 1,
 0,
 10,
 9

In [62]:
data_path = "Data/web-Google.txt"
with open(data_path, encoding="utf-8") as f:
        tab = f.readlines()
list_edges = [(int(a.split('\t')[0]),int(a.split('\t')[1].split('\n')[0])) for a in tab[4:]]  
# the first 4 lines are explanations. each line contains 'val_A\tval_B\n". 
# val_A is reachable by spliting
# val_B is reachable by spliting by \t and \n (working for last index)

In [125]:
data_path = "Data/p2p-Gnutella08.txt"
with open(data_path, encoding="utf-8") as f:
        tab = f.readlines()
list_edges_2 = [(int(a.split('\t')[0]),int(a.split('\t')[1].split('\n')[0])) for a in tab[4:]]  
# the first 4 lines are explanations. each line contains 'val_A\tval_B\n". 
# val_A is reachable by spliting
# val_B is reachable by spliting by \t and \n (working for last index)

In [66]:
list_edges

[(0, 11342),
 (0, 824020),
 (0, 867923),
 (0, 891835),
 (11342, 0),
 (11342, 27469),
 (11342, 38716),
 (11342, 309564),
 (11342, 322178),
 (11342, 387543),
 (11342, 427436),
 (11342, 538214),
 (11342, 638706),
 (11342, 645018),
 (11342, 835220),
 (11342, 856657),
 (11342, 867923),
 (11342, 891835),
 (824020, 0),
 (824020, 91807),
 (824020, 322178),
 (824020, 387543),
 (824020, 417728),
 (824020, 438493),
 (824020, 500627),
 (824020, 535748),
 (824020, 695578),
 (824020, 867923),
 (824020, 891835),
 (867923, 0),
 (867923, 11342),
 (867923, 136593),
 (867923, 414038),
 (867923, 500627),
 (867923, 523684),
 (867923, 760842),
 (867923, 815602),
 (867923, 835220),
 (867923, 846213),
 (867923, 857527),
 (867923, 891835),
 (891835, 0),
 (891835, 11342),
 (891835, 112028),
 (891835, 235849),
 (891835, 302284),
 (891835, 417728),
 (891835, 451592),
 (891835, 693969),
 (891835, 857527),
 (891835, 867923),
 (1, 53051),
 (1, 203402),
 (1, 223236),
 (1, 276233),
 (1, 552600),
 (1, 569212),
 (1, 635

In [72]:
from tqdm import tqdm

In [103]:
graph_data_dict={}
for edge in tqdm(list_edges):
    if(edge[0] not in graph_data_dict.keys()):
        graph_data_dict[edge[0]]= set({})
    graph_data_dict[edge[0]].add(edge[1])

100%|███████████████████████████████████████████████████████████████████| 5105039/5105039 [00:03<00:00, 1507828.68it/s]


In [99]:
graph_data = Graph()
for edge in tqdm(list_edges):
    A,B = edge[0],edge[1]
    graph_data.add_Node(A)
    graph_data.add_Node(B)
    graph_data.add_Edge(A,B)    

100%|████████████████████████████████████████████████████████████████████| 5105039/5105039 [00:06<00:00, 746090.24it/s]


In [139]:
graph_data=Graph()
for edge in tqdm(list_edges):
    A,B = edge[0], edge[1]
    if(A not in graph_data.NodesSet):
        graph_data.add_Node(A)
    if(B not in graph_data.NodesSet):
        graph_data.add_Node(B)
    graph_data.add_Edge(A,B)

100%|████████████████████████████████████████████████████████████████████| 5105039/5105039 [00:09<00:00, 560357.59it/s]


In [140]:
graph_data_2=Graph()
for edge in tqdm(list_edges_2):
    A,B = edge[0], edge[1]
    if(A not in graph_data_2.NodesSet):
        graph_data_2.add_Node(A)
    if(B not in graph_data_2.NodesSet):
        graph_data_2.add_Node(B)
    graph_data_2.add_Edge(A,B)

100%|████████████████████████████████████████████████████████████████████████| 20777/20777 [00:00<00:00, 630768.29it/s]


In [141]:
graph_data_2.size()

(6301, 20777)

In [101]:
a={}

In [ ]:
a.

In [100]:
help(graph_data.size)

Help on method size in module __main__:

size() method of __main__.Graph instance
    return (number of nodes, number of edges)



In [96]:
len(graph_data.NodesSet),len(graph_data.EdgesSet)

(875713, 5105039)

In [97]:
graph_data.NodesSet

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 99,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 1

In [84]:
for i in range(10):
    print(hash(list(graph_data.NodesSet)[i]))

0
0
0
0
0
0
0
0
1
1


## Algorithm

In [122]:
import numpy as np
import scipy.integrate as integrate
import time

In [ ]:
b=

In [4]:
'{0:b}'.format(2)


'10'

In [6]:
a='{0:b}'.format(2)

In [8]:
int(a,2)

2

In [261]:
'{0:b}'.format(hash("1"))

'-1100110100000011101100110010000110010111000001001101011010111'

In [299]:
class Hyperloglog:
    def __init__(self,b):
        self.b = b
        self.p = 2**b
        self.M=np.array([-np.inf]*self.p)
        self.alpha = self.__compute_alpha(self.p)  # compute alpha_p. Formula given in "HyperLogLog: the analysis of a near-optimal cardinality estimation algorithm" article        
    
    def __compute_alpha(self,p):
        def to_integrate(x, p):
            return (np.log2((2+x)/(1+x)))**p
        return 1/(p*integrate.quad(to_integrate,0,np.inf, args=(p))[0])
        
    def __zero_count(self,string):
        for j,a in enumerate(string[::-1]):
            if (int(a)==1):
                return 1+j
        return 0 
    
    def add(self,x):
        bin_x = '{0:b}'.format(x) #convert x to binary
        if len(bin_x)<self.b:
            bin_x = '0'*(self.b-len(bin_x)) + bin_x
        h_left = bin_x[:-self.b] # take b digits on the right size
        h_right = bin_x[-self.b:]        #take all other digits
        i = int(h_right,2)       #convert binary to i
        self.M[i] = max(self.M[i],self.__zero_count(h_left))
        
    def size(self):
        z=0
        for a in self.M:
            z+=2**(-a)
        return self.alpha*(self.p**2)*1/z
    
    def union(self,other : Hyperloglog) -> Hyperloglog:
        if (self.b!=other.b):
            raise Exception("Both Hyperloglog should have the same b")
        res = Hyperloglog(self.b)
        for (val_other,val_self,i) in zip(self.M, other.M, range(self.p)):
            res.M[i] = max(val_other,val_self)
        return res

In [262]:
max(graph_data_2.NodesSet)

6300

In [263]:
'{0:b}'.format(6300)

'1100010011100'

In [300]:
test = Hyperloglog(5)

In [301]:
for i in range(25):
    test.add(i)

In [291]:
a = "1000011"

In [294]:
for b in a[::-1]:
    print(b)

1
1
0
0
0
0
1


In [290]:
test.M

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [257]:
test.M

array([0., 0., 0., 0., 2., 2., 2., 1.])

In [302]:
test.size()

0.0

## Hyperball

In [111]:
True&False

False

In [119]:
for a in tqdm(range(len(graph_data_2.NodesSet))):
    c = Hyperloglog(10)

  4%|███                                                                      | 37072/875713 [00:14<05:31, 2529.79it/s]


KeyboardInterrupt: 

In [303]:
def Hyperball(b: int,graph: Graph):
    t_0 = time.time()
    n = len(graph.NodesSet)
    print("Generation of array of Hyperloglog")
    c = np.array([Hyperloglog(b) for i in range(n)])
    print("End of generation.\t Computation Time:",round(time.time()-t_0,2),"s")
    t = 1
    c_t_minus_1 = np.array([Hyperloglog(b) for i in range(n)])                     
    #c_t_minus_1 is the value of c for the step (t-1) while we compute step t
    #thus we can modify the value of c easily since we have saved previous values
    coreachable = np.zeros(n)
    sum_distance = np.zeros(n)
    sum_inv_distance = np.zeros(n)
    zero_change = False
    for i in range(n):
        c[i].add(i)
    while (not zero_change):
        t_0 = time.time()
        zero_change = True
        for i in range(n):
            for dest in list(graph.EdgesDict[i]):
                c[i] = c[i].union(c_t_minus_1[dest])
            estimation_numb = c[i].size()-c_t_minus_1[i].size()
            coreachable[i]+=estimation_numb
            sum_distance[i] += t*estimation_numb
            sum_inv_distance[i] += 1/t*estimation_numb
            zero_change = (zero_change & (estimation_numb==0))    # if there is a change, zero_change becomes False
        print(t,"\tComputation time of the step:", round(time.time()-t_0,2),"s")
        t=t+1
    return coreachable,sum_distance, sum_inv_distance

In [158]:
(i in graph_data_2)

TypeError: argument of type 'Graph' is not iterable

In [159]:
a=True
for i in range(len(graph_data_2.NodesSet)):
    a = a & (i in graph_data_2.NodesSet)

In [160]:
a

True

In [218]:
i=0
n = len(graph_data_2.NodesSet)
c = np.array([Hyperloglog(10) for i in range(len(graph_data_2.NodesSet))])
zero_change = True
t=1
for j in range(n):
    c[j].add(j)

init = c[i].size()
for dest in list(graph_data_2.EdgesDict[i]):
    print(dest)
    print(c[dest].M)
    c[i] = c[i].union(c[dest])
estimation_numb = c[i].size() - init
coreachable = estimation_numb
sum_distance = t*estimation_numb
sum_inv_distance = 1/t*estimation_numb
zero_change = (zero_change & (estimation_numb==0))    # if there is a change, zero_change becomes False


1
[-inf   0. -inf ... -inf -inf -inf]
2
[-inf -inf   0. ... -inf -inf -inf]
3
[-inf -inf -inf ... -inf -inf -inf]
4
[-inf -inf -inf ... -inf -inf -inf]
5
[-inf -inf -inf ... -inf -inf -inf]
6
[-inf -inf -inf ... -inf -inf -inf]
7
[-inf -inf -inf ... -inf -inf -inf]
8
[-inf -inf -inf ... -inf -inf -inf]
9
[-inf -inf -inf ... -inf -inf -inf]
10
[-inf -inf -inf ... -inf -inf -inf]


In [225]:
c[0].alpha

0.720587225976452

In [224]:
c[i].size()

0.0

In [194]:
list(graph_data_2.EdgesDict[i])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [196]:
c[0].M,c[10].M

(array([  0., -inf, -inf, ..., -inf, -inf, -inf]),
 array([  0., -inf, -inf, ..., -inf, -inf, -inf]))

In [193]:
for dest in list(graph_data_2.EdgesDict[i]):
    print(i)
    d = c[i].union(c[dest])

0
0
0
0
0
0
0
0
0
0


In [219]:
coreachable, sum_distance, sum_inv_distance

(0.0, 0.0, 0.0)

In [304]:
a,b,c = Hyperball(10, graph_data_2)

Generation of array of Hyperloglog
End of generation.	 Computation Time: 2.23 s
1 	Computation time of the step: 22.77 s


In [305]:
a,b,c

(array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]))